In [1]:
import time
from datetime import datetime

import torch
import torch.nn.functional as F
import torch.nn as nn

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
GPU = 0
device = torch.device("cuda:{}".format(GPU)) if torch.cuda.is_available() else torch.device("cpu")

In [2]:
# Param list
EPOCH = 300
TIMESTEP = 12
TRAINDAYS = 30

OPTIMIZER = 'Adam'
LEARN = 0.0001
PATIENCE = 20
beijing = None
changchun = None
shenzhen = None
shanghai = None
Loss = nn.L1Loss()
scaler_dict = dict()
model_dict = dict()

In [3]:
def csv_to_tensor(file):
    Data = pd.read_csv(file)
    scaler = StandardScaler()
    scaler_dict[file.split('/')[2][0:-9]] = scaler
    f = Data['confirmedNum'].to_numpy()[::-1].reshape(42, 1).astype(float)
    f = scaler.fit_transform(f)
    x_data = np.arange(0, 42, 1)
    x_data = x_data.reshape(-1, 1).astype(int)
    data = np.append(x_data, values=f, axis=1)
    d = torch.from_numpy(data.copy())
    return d


In [4]:
def read_csv_to_list():
    global beijing
    global shenzhen
    global changchun
    global shanghai
    beijing = csv_to_tensor('./input/beijing_data.csv')
    shenzhen = csv_to_tensor('./input/shenzhen_data.csv')
    changchun = csv_to_tensor('./input/changchun_data.csv')
    shanghai = csv_to_tensor('./input/shanghai_data.csv')


read_csv_to_list()

In [5]:
shenzhen

tensor([[ 0.0000e+00, -1.3168e+00],
        [ 1.0000e+00, -1.2994e+00],
        [ 2.0000e+00, -1.2835e+00],
        [ 3.0000e+00, -1.2739e+00],
        [ 4.0000e+00, -1.2517e+00],
        [ 5.0000e+00, -1.2184e+00],
        [ 6.0000e+00, -1.1961e+00],
        [ 7.0000e+00, -1.1739e+00],
        [ 8.0000e+00, -1.1358e+00],
        [ 9.0000e+00, -1.0913e+00],
        [ 1.0000e+01, -1.0532e+00],
        [ 1.1000e+01, -9.9766e-01],
        [ 1.2000e+01, -9.4685e-01],
        [ 1.3000e+01, -8.4999e-01],
        [ 1.4000e+01, -7.7060e-01],
        [ 1.5000e+01, -6.4198e-01],
        [ 1.6000e+01, -5.4671e-01],
        [ 1.7000e+01, -3.5935e-01],
        [ 1.8000e+01, -2.2438e-01],
        [ 1.9000e+01, -1.0211e-01],
        [ 2.0000e+01, -1.7958e-02],
        [ 2.1000e+01,  6.1435e-02],
        [ 2.2000e+01,  1.1066e-01],
        [ 2.3000e+01,  1.6782e-01],
        [ 2.4000e+01,  2.3927e-01],
        [ 2.5000e+01,  3.1549e-01],
        [ 2.6000e+01,  4.3775e-01],
        [ 2.7000e+01,  5.981

In [6]:
class CausalConv1d(torch.nn.Conv1d):
    def __init__(self,
                 in_channels,
                 out_channels,
                 kernel_size,
                 stride=1,
                 dilation=1,
                 groups=1,
                 bias=True):
        super(CausalConv1d, self).__init__(
            in_channels,
            out_channels,
            kernel_size=kernel_size,
            stride=stride,
            padding=0,
            dilation=dilation,
            groups=groups,
            bias=bias)

        self.__padding = (kernel_size - 1) * dilation

    def forward(self, input):
        return super(CausalConv1d, self).forward(F.pad(input, (self.__padding, 0)))


class context_embedding(torch.nn.Module):
    def __init__(self, in_channels=1, embedding_size=256, k=5):
        super(context_embedding, self).__init__()
        self.causal_convolution = CausalConv1d(in_channels, embedding_size, kernel_size=k)

    def forward(self, x):
        x = self.causal_convolution(x)
        return torch.tanh(x)


class Transformer(nn.Module):
    def __init__(self, device=device, dmodel=256):
        super(Transformer, self).__init__()
        self.input_embedding = context_embedding(2, dmodel, 9)
        self.positional_embedding = torch.nn.Embedding(TIMESTEP, dmodel)
        self.device = device
        self.dmodel = dmodel
        self.decode_layer = torch.nn.TransformerEncoderLayer(d_model=dmodel, nhead=4)
        self.transformer_decoder = torch.nn.TransformerEncoder(self.decode_layer, num_layers=6)
        self.fc1 = torch.nn.Linear(dmodel, int(dmodel / 2))
        self.fc12 = torch.nn.Linear(int(dmodel / 2), 1)
        self.fc2 = torch.nn.Linear(TIMESTEP, 1)

    def forward(self, x, y, attention_mask):
        z = torch.cat((y.unsqueeze(1), x.unsqueeze(1)), 1)
        z_embedding = self.input_embedding(z).permute(2, 0, 1)
        positional_embeddings = self.positional_embedding(torch.arange(0, TIMESTEP).to(self.device)).expand(1, TIMESTEP,
                                                                                                            self.dmodel).permute(
            1, 0, 2)
        input_embedding = z_embedding + positional_embeddings
        transformer_embedding = self.transformer_decoder(input_embedding, attention_mask)

        output = self.fc1(transformer_embedding.permute(1, 0, 2))
        output = self.fc12(output).permute(2, 0, 1)
        output = self.fc2(output)
        return output

In [7]:
def evaluate_epoch(model, name,test):
    model.eval()
    l_sum, n = 0.0, 0
    with torch.no_grad():
        for i in range(0, TIMESTEP):
            x = test[i + TRAINDAYS - TIMESTEP: i + TRAINDAYS, 0].reshape(1, -1).float()
            y = test[i + TRAINDAYS - TIMESTEP: i + TRAINDAYS, 1].reshape(1, -1).float()
            attention_mask = torch.from_numpy(np.zeros((TIMESTEP, TIMESTEP)))
            y_pred = model(x.cuda(), y.cuda(), attention_mask.cuda())[0][0][0]
            # y_pred = model(x.cuda())[0][0]
            loss = Loss(y_pred.float(), test[i + TRAINDAYS][1].cuda().float())
            l_sum += loss.item() * y.shape[0]
            n += y.shape[0]
        return l_sum / n


In [8]:
def train(model, name, data):

    print('Model Training Started ...', time.ctime())
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARN)
    min_val_loss = np.inf
    for epoch in range(EPOCH):
        starttime = datetime.now()
        model.train()
        loss_sum, n = 0.0, 0
        for i in range(0, TRAINDAYS - TIMESTEP):
            optimizer.zero_grad()
            x = data[i:i + TIMESTEP, 0].reshape(1, -1).float()
            y = data[i:i + TIMESTEP, 1].reshape(1, -1).float()
            attention_mask = torch.from_numpy(np.zeros((TIMESTEP, TIMESTEP)))
            y_pred = model(x.cuda(), y.cuda(), attention_mask.cuda())[0][0][0]
            y = data[i + TIMESTEP][1]
            loss = Loss(y_pred.float(), y.cuda().float())
            loss.backward()
            optimizer.step()
            loss_sum += loss.item()
            n += 1
        train_loss = loss_sum / n
        val_loss = evaluate_epoch(model,name, data)
        if val_loss < min_val_loss:
            wait = 0
            min_val_loss = val_loss
            open('./model/' + name + '.pt', 'a')
            torch.save(model.state_dict(), './model/' + name + '.pt')
        else:
            wait += 1
            if wait == PATIENCE:
                print('Early stopping at epoch: %d' % epoch)
                break
        endtime = datetime.now()
        epoch_time = (endtime - starttime).seconds
        print("epoch", epoch, "time used:", epoch_time, " seconds ", "train loss:", train_loss, "validation loss:",
              val_loss)
    print('Model Training Ended ...', time.ctime())

In [9]:
import Metrics


def test_model(model, name, data):
    loss = nn.L1Loss()
    print('Model Testing Started ...', time.ctime())
    model.load_state_dict(torch.load('./model/' + name + '.pt'))
    YS = data[30:42, 1]
    YS_pred = []
    scaler = scaler_dict[name]
    with torch.no_grad():
        for i in range(0, TIMESTEP):
            x = data[i + TRAINDAYS - TIMESTEP: i + TRAINDAYS, 0].reshape(1, -1).float()
            y = data[i + TRAINDAYS - TIMESTEP: i + TRAINDAYS, 1].reshape(1, -1).float()
            attention_mask = torch.from_numpy(np.zeros((TIMESTEP, TIMESTEP)))
            y_pred = model(x.cuda(), y.cuda(), attention_mask.cuda())[0][0][0]
            YS_pred.append(y_pred.cpu())
    YS_pred = np.array(YS_pred)
    YS = scaler.inverse_transform(YS.reshape(-1,1))
    YS_pred = scaler.inverse_transform(YS_pred.reshape(-1,1))
    Mae = 0
    for i in range(TIMESTEP):
        MSE, RMSE, MAE, MAPE = Metrics.evaluate(YS[i], YS_pred[i])
        Mae += MAE
        print("%d step, %s, %s, MSE, RMSE, MAE, MAPE, %.10f, %.10f, %.10f, %.10f" % (
            i + 1, name, 'Transformer', MSE, RMSE, MAE, MAPE))
    print("Average MAE Loss is %.10f" % (Mae/TIMESTEP))
    print('Model Testing Ended ...', time.ctime())


In [10]:
# train every city
model = Transformer().cuda(device=device)
model_dict['shenzhen'] = model
train(model,'shenzhen',shenzhen)

Model Training Started ... Mon May 23 02:33:18 2022
epoch 0 time used: 3  seconds  train loss: 0.2605787383185493 validation loss: 0.7255209485689799
epoch 1 time used: 0  seconds  train loss: 0.6748033124539587 validation loss: 0.9631207883358002
epoch 2 time used: 0  seconds  train loss: 0.5187350780599647 validation loss: 1.1635419676701229
epoch 3 time used: 0  seconds  train loss: 0.46484264276093906 validation loss: 1.186877816915512
epoch 4 time used: 0  seconds  train loss: 0.4424713109102514 validation loss: 1.1791003147761028
epoch 5 time used: 0  seconds  train loss: 0.459691612670819 validation loss: 1.1734035809834797
epoch 6 time used: 0  seconds  train loss: 0.46485449249545735 validation loss: 1.2082143028577168
epoch 7 time used: 0  seconds  train loss: 0.4524774248194363 validation loss: 1.2047715733448665
epoch 8 time used: 0  seconds  train loss: 0.4550346868733565 validation loss: 1.2004384547472
epoch 9 time used: 0  seconds  train loss: 0.42198105063289404 valida

In [11]:
model = Transformer().cuda(device=device)
model_dict['shanghai'] = model
train(model,'shanghai',shanghai)

Model Training Started ... Mon May 23 02:33:41 2022
epoch 0 time used: 0  seconds  train loss: 0.22725348878237936 validation loss: 0.33172592520713806
epoch 1 time used: 0  seconds  train loss: 0.9830518133110471 validation loss: 1.179468368490537
epoch 2 time used: 0  seconds  train loss: 0.6465969946649339 validation loss: 1.3109981268644333
epoch 3 time used: 0  seconds  train loss: 0.6026915903720591 validation loss: 1.282351404428482
epoch 4 time used: 0  seconds  train loss: 0.6046084687113762 validation loss: 1.2708813548088074
epoch 5 time used: 0  seconds  train loss: 0.5961640073607365 validation loss: 1.2510628700256348
epoch 6 time used: 0  seconds  train loss: 0.5920422017160389 validation loss: 1.2429760893185933
epoch 7 time used: 0  seconds  train loss: 0.6185108640541633 validation loss: 1.3386391699314117
epoch 8 time used: 0  seconds  train loss: 0.5671452490819825 validation loss: 1.2560378462076187
epoch 9 time used: 0  seconds  train loss: 0.5533341216958232 vali

In [12]:
model = Transformer().cuda(device=device)
model_dict['beijing'] = model
train(model,'beijing',beijing)

Model Training Started ... Mon May 23 02:34:15 2022
epoch 0 time used: 0  seconds  train loss: 0.43305765754646725 validation loss: 1.5917422870794933
epoch 1 time used: 0  seconds  train loss: 0.5002476531598303 validation loss: 1.5987942516803741
epoch 2 time used: 0  seconds  train loss: 0.4968881706396739 validation loss: 1.6185342172781627
epoch 3 time used: 0  seconds  train loss: 0.48348354051510495 validation loss: 1.6322675347328186
epoch 4 time used: 0  seconds  train loss: 0.4791332433621089 validation loss: 1.6422257622083027
epoch 5 time used: 0  seconds  train loss: 0.49308429741197163 validation loss: 1.6606476604938507
epoch 6 time used: 0  seconds  train loss: 0.4838179623087247 validation loss: 1.636814723412196
epoch 7 time used: 0  seconds  train loss: 0.4595336682266659 validation loss: 1.4598620235919952
epoch 8 time used: 0  seconds  train loss: 0.39715105295181274 validation loss: 0.3251331200202306
epoch 9 time used: 0  seconds  train loss: 0.524996693763468 va

In [13]:
model = Transformer().cuda(device=device)
model_dict['changchun'] = model
train(model,'changchun',changchun)

Model Training Started ... Mon May 23 02:34:44 2022
epoch 0 time used: 0  seconds  train loss: 0.4417699740992652 validation loss: 0.6413323481877645
epoch 1 time used: 0  seconds  train loss: 0.47894033210145104 validation loss: 0.6167537321647009
epoch 2 time used: 0  seconds  train loss: 0.475407434006532 validation loss: 0.6312257250150045
epoch 3 time used: 0  seconds  train loss: 0.47777266717619365 validation loss: 0.6357981363932291
epoch 4 time used: 0  seconds  train loss: 0.46385550250609714 validation loss: 0.637995312611262
epoch 5 time used: 0  seconds  train loss: 0.45982586923572755 validation loss: 0.6385976523160934
epoch 6 time used: 0  seconds  train loss: 0.4724905830290582 validation loss: 0.628715326388677
epoch 7 time used: 0  seconds  train loss: 0.4525258036123382 validation loss: 0.529049813747406
epoch 8 time used: 0  seconds  train loss: 0.4617776564425892 validation loss: 0.2515124132235845
epoch 9 time used: 0  seconds  train loss: 0.34428588507903946 val

In [14]:
test_model(model_dict['shenzhen'],'shenzhen',shenzhen)

Model Testing Started ... Mon May 23 02:35:27 2022
1 step, shenzhen, Transformer, MSE, RMSE, MAE, MAPE, 10076.4394644648, 100.3814697266, 100.3814697266, 7.1091692441
2 step, shenzhen, Transformer, MSE, RMSE, MAE, MAPE, 2010.5349576622, 44.8389892578, 44.8389892578, 3.0113491778
3 step, shenzhen, Transformer, MSE, RMSE, MAE, MAPE, 60.0038595349, 7.7462158203, 7.7462158203, 0.5013731923
4 step, shenzhen, Transformer, MSE, RMSE, MAE, MAPE, 1987.4486690164, 44.5808105469, 44.5808105469, 2.7621320041
5 step, shenzhen, Transformer, MSE, RMSE, MAE, MAPE, 4394.0132361054, 66.2873535156, 66.2873535156, 4.0174153646
6 step, shenzhen, Transformer, MSE, RMSE, MAE, MAPE, 5970.1088531166, 77.2664794922, 77.2664794922, 4.6156797785
7 step, shenzhen, Transformer, MSE, RMSE, MAE, MAPE, 6641.0761016756, 81.4927978516, 81.4927978516, 4.8220590445
8 step, shenzhen, Transformer, MSE, RMSE, MAE, MAPE, 6445.1183014512, 80.2814941406, 80.2814941406, 4.7252203732
9 step, shenzhen, Transformer, MSE, RMSE, MAE,

In [15]:
test_model(model_dict['shanghai'],'shanghai',shanghai)

Model Testing Started ... Mon May 23 02:35:28 2022
1 step, shanghai, Transformer, MSE, RMSE, MAE, MAPE, 22700460.2500000000, 4764.5000000000, 4764.5000000000, 10.7730746620
2 step, shanghai, Transformer, MSE, RMSE, MAE, MAPE, 22573411.4751586914, 4751.1484375000, 4751.1484375000, 10.4379551771
3 step, shanghai, Transformer, MSE, RMSE, MAE, MAPE, 72840.9494628906, 269.8906250000, 269.8906250000, 0.5291246790
4 step, shanghai, Transformer, MSE, RMSE, MAE, MAPE, 24880.1330566406, 157.7343750000, 157.7343750000, 0.3018435329
5 step, shanghai, Transformer, MSE, RMSE, MAE, MAPE, 74413.8726196289, 272.7890625000, 272.7890625000, 0.5142500141
6 step, shanghai, Transformer, MSE, RMSE, MAE, MAPE, 204409.9512329102, 452.1171875000, 452.1171875000, 0.8407572060
7 step, shanghai, Transformer, MSE, RMSE, MAE, MAPE, 58326.0234985352, 241.5078125000, 241.5078125000, 0.4468312318
8 step, shanghai, Transformer, MSE, RMSE, MAE, MAPE, 12573.7676391602, 112.1328125000, 112.1328125000, 0.2064604737
9 step, 

In [16]:
test_model(model_dict['beijing'],'beijing',beijing)

Model Testing Started ... Mon May 23 02:35:28 2022
1 step, beijing, Transformer, MSE, RMSE, MAE, MAPE, 677.5910229422, 26.0305786133, 26.0305786133, 3.7562162501
2 step, beijing, Transformer, MSE, RMSE, MAE, MAPE, 82.1123131551, 9.0615844727, 9.0615844727, 1.2002098639
3 step, beijing, Transformer, MSE, RMSE, MAE, MAPE, 161.3073637486, 12.7006835938, 12.7006835938, 1.6282927684
4 step, beijing, Transformer, MSE, RMSE, MAE, MAPE, 874.4575270452, 29.5712280273, 29.5712280273, 3.6283715371
5 step, beijing, Transformer, MSE, RMSE, MAE, MAPE, 3119.4447585978, 55.8519897461, 55.8519897461, 6.5095559145
6 step, beijing, Transformer, MSE, RMSE, MAE, MAPE, 5105.4967385679, 71.4527587891, 71.4527587891, 8.0283998639
7 step, beijing, Transformer, MSE, RMSE, MAE, MAPE, 7439.8942798413, 86.2548217773, 86.2548217773, 9.3450511135
8 step, beijing, Transformer, MSE, RMSE, MAE, MAPE, 11107.1709728278, 105.3905639648, 105.3905639648, 10.9553600795
9 step, beijing, Transformer, MSE, RMSE, MAE, MAPE, 1551

In [17]:
test_model(model_dict['changchun'],'changchun',changchun)

Model Testing Started ... Mon May 23 02:35:28 2022
1 step, changchun, Transformer, MSE, RMSE, MAE, MAPE, 229892.1551551819, 479.4707031250, 479.4707031250, 2.0893790445
2 step, changchun, Transformer, MSE, RMSE, MAE, MAPE, 207711.6230621338, 455.7539062500, 455.7539062500, 1.9790434072
3 step, changchun, Transformer, MSE, RMSE, MAE, MAPE, 118985.9211769104, 344.9433593750, 344.9433593750, 1.4866966614
4 step, changchun, Transformer, MSE, RMSE, MAE, MAPE, 18395.2002105713, 135.6289062500, 135.6289062500, 0.5774637299
5 step, changchun, Transformer, MSE, RMSE, MAE, MAPE, 8843.7126197815, 94.0410156250, 94.0410156250, 0.3980572090
6 step, changchun, Transformer, MSE, RMSE, MAE, MAPE, 2237.5486793518, 47.3027343750, 47.3027343750, 0.1989014144
7 step, changchun, Transformer, MSE, RMSE, MAE, MAPE, 386.4449501038, 19.6582031250, 19.6582031250, 0.0822278125
8 step, changchun, Transformer, MSE, RMSE, MAE, MAPE, 2289.5851173401, 47.8496093750, 47.8496093750, 0.1989506024
9 step, changchun, Tran

In [18]:
def predict_day(model, name,days):
    model.eval()
    with torch.no_grad():
        x = days[:,0].reshape(1, -1).float()
        y = days[:,1].reshape(1, -1).float()
        attention_mask = torch.from_numpy(np.zeros((TIMESTEP, TIMESTEP)))
        y_pred = model(x.cuda(), y.cuda(), attention_mask.cuda())[0][0][0]
        # Does not output values that defy common sense.
        if y_pred < y[0,-1]:
            y_pred = y[0,-1]
        return scaler_dict[name].inverse_transform(y_pred.cpu().reshape(-1,1))[0][0]

In [23]:
# predict_day(model_dict['shenzhen'],'shenzhen',shenzhen[18:18+TIMESTEP])
predict_one_city = {} # 这个用来存储每个城市的预测数据, 对于每个城市都预测第31天到第42天的情况
predict_one_city['shenzhen'] = []
predict_one_city['changchun'] =[]
predict_one_city['shanghai'] = []
predict_one_city['beijing'] = []
for i in range(0,12): 
    predict_one_city['shenzhen'].append(predict_day(model_dict['shenzhen'],'shenzhen',shenzhen[18+i:30+i]))
for i in range(0,12): 
    predict_one_city['changchun'].append(predict_day(model_dict['changchun'],'changchun',changchun[18+i:30+i]))
for i in range(0,12): 
    predict_one_city['shanghai'].append(predict_day(model_dict['shanghai'],'shanghai',shanghai[18+i:30+i]))
for i in range(0,12):
    predict_one_city['beijing'].append(predict_day(model_dict['beijing'],'beijing',beijing[18+i:30+i]))
predict_one_city

1512.3815044144976

In [19]:
predict_another_city = {} # 这个预测使用不同城市的模型来体现不同城市的防疫效果
predict_another_city['sz_to_sh'] = []  #这个时候预测第13到第42天情况
predict_another_city['sh_to_sz'] = []
predict_another_city['sz_to_cc'] = []
predict_another_city['sz_to_bj'] = []
# 使用深圳的模型来预测上海
for i in range(0,30): 
    predict_another_city['sz_to_sh'].append(predict_day(model_dict['shenzhen'],'shenzhen',shanghai[0+i:12+i]))
# 使用上海的模型来预测深圳
for i in range(0,30): 
    predict_another_city['sh_to_sz'].append(predict_day(model_dict['shanghai'],'shanghai',shenzhen[0+i:12+i]))
# 使用深圳的模型来预测长春
for i in range(0,30): 
    predict_another_city['sz_to_cc'].append(predict_day(model_dict['shenzhen'],'shenzhen',changchun[0+i:12+i]))
# 使用深圳的模型来预测北京
for i in range(0,30): 
    predict_another_city['sz_to_bj'].append(predict_day(model_dict['shenzhen'],'shenzhen',beijing[0+i:12+i]))   
predict_another_city